In [4]:
!pip install swig

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    --------------------------------------- 0.0/2.6 MB 393.8 kB/s eta 0:00:07
    --------------------------------------- 0.1/2.6 MB 550.5 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.6 MB 585.1 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/2.6 MB 919.0 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.6 MB 892.5 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.6 MB 983.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.6 MB 983.9 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.6 MB 905.4 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.6 MB 905.4 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.6 MB 905.4 kB/s eta 0:00:03
   ---- -----


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Alaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
!pip install smac

  Using cached smac-2.1.0-py3-none-any.whl
  Using cached pynisher-1.0.10-py3-none-any.whl
  Using cached pyrfr-0.9.0.tar.gz (295 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dask-2024.6.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached dask_jobqueue-0.8.5-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached emcee-3.1.6-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached distributed-2024.6.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached partd-1.4.2-py3-none-any.whl.metadata (4.6 kB)
  Using cached locket-1.0.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached msgpack-1.0.8-cp311-cp311-win_amd64.whl.metadata (9.4 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached tblib-3.0.0-py3-none-any.whl.metadata (25 kB)
  Using cached zict-3.0.0-py2.py3-none-any.whl.metadata (899 bytes)
Using cached emcee-3.1.6-py2.py3-none-any.whl (47 kB)
Using cached dask_jobqueue-0


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Alaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install ConfigSpace

  Using cached ConfigSpace-0.7.1-cp311-cp311-win_amd64.whl.metadata (7.4 kB)
  Using cached more_itertools-10.3.0-py3-none-any.whl.metadata (36 kB)
Using cached ConfigSpace-0.7.1-cp311-cp311-win_amd64.whl (2.2 MB)
Using cached more_itertools-10.3.0-py3-none-any.whl (59 kB)



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Alaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from ConfigSpace import Configuration, ConfigurationSpace

import numpy as np
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

iris = datasets.load_iris()


def train(config: Configuration, seed: int = 0) -> float:
    classifier = SVC(C=config["C"], random_state=seed)
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
#     print (np.mean(scores))
    return 1 - np.mean(scores)


configspace = ConfigurationSpace({"C": (0.100, 1000.0)})

# Scenario object specifying the optimization environment
scenario = Scenario(configspace, deterministic=True, n_trials=200)

# Use SMAC to find the best configuration/hyperparameters
smac = HyperparameterOptimizationFacade(scenario, train)
incumbent = smac.optimize()

[INFO][abstract_initial_design.py:147] Using 10 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[INFO][abstract_intensifier.py:515] Added config 8a6596 as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:594] Added config a3ff11 and rejected config 8a6596 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 0ce248 and rejected config a3ff11 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 3985dc and rejected config 0ce248 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 78dba7 and rejected config 3985dc as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 50 trials.
[INFO][smbo.py:319] Finishe

In [7]:
incumbent.values


<bound method Mapping.values of Configuration(values={
  'C': 7.082646102284318,
})>

In [8]:
classifier = SVC(C=6.724148702862232)
scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
print(np.mean(scores))

0.9866666666666667


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, ShuffleSplit
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
# import seaborn as sn
import warnings

warnings.filterwarnings('ignore')

[INFO][utils.py:148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO][utils.py:160] NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'seaborn'

In [10]:
# Load the Credit Card Fraud Detection dataset
url = "https://www.openml.org/data/get_csv/31/dataset_31.csv"
df = pd.read_csv(url)

# Drop missing values
df = df.dropna()

# Convert categorical features to numerical
df = pd.get_dummies(df, drop_first=True)

In [11]:
df.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status_'<0',checking_status_'>=200',checking_status_'no checking',...,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_'unemp/unskilled non res',job_'unskilled resident',job_skilled,own_telephone_yes,foreign_worker_yes,class_good
0,6,1169,4,4,67,2,1,True,False,False,...,True,False,True,False,False,False,True,True,True,True
1,48,5951,2,2,22,1,1,False,False,False,...,True,False,True,False,False,False,True,False,True,False
2,12,2096,2,3,49,1,2,False,False,True,...,True,False,True,False,False,True,False,False,True,True
3,42,7882,2,4,45,1,2,True,False,False,...,True,False,False,False,False,False,True,False,True,True
4,24,4870,3,4,53,2,2,True,False,False,...,True,False,False,False,False,False,True,False,True,False


In [12]:
df['class_good'].value_counts()

class_good
True     700
False    300
Name: count, dtype: int64

In [13]:
target = "class_good"
X = df.drop(columns=target)
y = df[target]

# Split the dataset into training and testing sets
X_train_raw, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.feature_selection import SelectKBest, f_classif

def Which_features(X_train,y_train,number_of_columns):

    # Select the top k features based on ANOVA F-statistic
    selector = SelectKBest(score_func=f_classif, k=number_of_columns)
    X_train_selected = selector.fit_transform(X_train, y_train)

    # Get the column names of the selected features
    selected_feature_names = X_train.columns[selector.get_support()]

    
    return list(selected_feature_names)

selected_features = Which_features(X_train_raw,y_train,number_of_columns=15)
selected_features

['duration',
 'credit_amount',
 'age',
 "checking_status_'<0'",
 "checking_status_'no checking'",
 "credit_history_'critical/other existing credit'",
 "credit_history_'no credits/all paid'",
 "purpose_'new car'",
 'purpose_radio/tv',
 "savings_status_'<100'",
 "property_magnitude_'no known property'",
 "property_magnitude_'real estate'",
 'other_payment_plans_none',
 'housing_own',
 'housing_rent']

In [15]:
X =X[selected_features]
X_train_raw = X_train_raw[selected_features]
X_test = X_test[selected_features]

In [16]:
X_train_raw.shape

(800, 15)

In [17]:
classifer=KNeighborsClassifier(n_neighbors=7)
classifer.fit(X_train_raw,y_train)
y_pred = classifer.predict(X_test)
loss=1-accuracy_score(y_test,y_pred)
print(loss)


0.32999999999999996


In [18]:
def plot_from_smac(smac) -> None:
    plt.figure()
    configs = smac.runhistory.get_configs()
    incumbents = smac.intensifier.get_incumbents()

    for i, config in enumerate(configs):
        if config in incumbents:
            continue

        label = None
        if i == 0:
            label = "Configuration"

        x = config["x"]
        f1, f2 = mymodell(x)
        plt.scatter(f1, f2, c="blue", alpha=0.1, marker="o", zorder=3000, label=label)

    for i, config in enumerate(incumbents):
        label = None
        if i == 0:
            label = "Incumbent"

        x = config["x"]
        f1, f2 = mymodell(x)
        plt.scatter(f1, f2, c="red", alpha=1, marker="x", zorder=3000, label=label)

    plt.xlabel("f1")
    plt.ylabel("f2")
    plt.title("Schaffer 2D")
    plt.legend()

    plt.show()

In [21]:
!pip install statsmodels

  Using cached patsy-0.5.6-py2.py3-none-any.whl.metadata (3.5 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB 660.6 kB/s eta 0:00:15
   ---------------------------------------- 0.1/9.9 MB 656.4 kB/s eta 0:00:15
   ---------------------------------------- 0.1/9.9 MB 1.0 MB/s eta 0:00:10
    --------------------------------------- 0.2/9.9 MB 1.5 MB/s eta 0:00:07
    --------------------------------------- 0.2/9.9 MB 1.4 MB/s eta 0:00:07
    --------------------------------------- 0.2/9.9 MB 1.4 MB/s eta 0:00:07
   - -------------------------------------- 0.3/9.9 MB 983.0 kB/s eta 0:00:10
   - -------------------------------------- 0.3/9.9 MB 1.1 MB/s eta 0:00:10
   - -------------------------------------- 0.3/9.9 MB 1.1 MB/s eta 0:00:09
   - -------------------------------------- 0.3/9.9 MB 1.1 MB/s eta 0:00:09
   - -------------------------


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Alaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [53]:
from statsmodels.tsa.arima.model import ARIMA
def Arima(train,test, p, d, q,freq):
    train = train.asfreq(freq)
    test = test.asfreq(freq)
    
    
    model_fit = ARIMA(train['y'], order=(p, d, q))
    model_fit = model_fit.fit()
    forecast_values = model_fit.forecast(steps=len(test))
        

    return  np.mean(np.abs((forecast_values - test['y']) / test['y'])) * 100




In [54]:
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from ConfigSpace.conditions import InCondition,EqualsCondition
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from smac import Callback
import time
#potential optimizations
#add timeserie
#search for the best hypers parameters for the each model
#add random state parameters
#what is the problem with linear svc
class CustomCallback(Callback):
    def __init__(self):
        self.trials_counter=0
    def on_start(self, smbo):
        print("let's start the optimization")
    def on_tell_end (self, smbo,info, value):
        self.trials_counter+=1
        print(f"the number of trials is: {self.trials_counter}")
        if self.trials_counter%10==0:
            incumbents = smbo.intensifier.get_incumbents()
            for incumbent in incumbents:
                print(f"the incumbent is: {incumbent.get_dictionary()}")
                print(f"the incumbent loss is: {smbo.runhistory.get_cost(incumbent)}")
            if self.trials_counter==100:
                print("let's stop the optimization at trial 100")
                return False
            return None
class Models:
    def __init__(self,similarModels,problemType,X_train,Y_train,X_test,Y_test,freq='d',m=7):
        self.Models=similarModels
        self.Problemtype=problemType
        self.X_train=X_train
        self.Y_train=Y_train
        self.X_test=X_test
        self.Y_test=Y_test
        self.freq=freq
        self.m=m
    def configspace(self):
        confs = ConfigurationSpace(seed=0)
        #HPOs
        if self.Problemtype=='Classification':
            # models are ['KNN','LR',"RF",'SVC']
            models=Categorical('Models',self.Models)
            #KNN parameters
            Kneighbors=Integer('Ks',(1,10),default=1)
            #LR  and svc Parameters
            rc=Float('regularizationStre',(0.01,1))
            #RF parameters
            nestimators=Integer('n_estimators',(1,20),default=10)
            #SVC parameters
            kernel=Categorical('kernel',['linear','rbf'])
            #dependencies

            useks=InCondition(child=Kneighbors,parent=models,values=['KNN'])
            userc=InCondition(child=rc,parent=models,values=['LR','SVC'])
            usekernel=InCondition(child=kernel,parent=models,values=['SVC'])
            useEst=EqualsCondition(child=nestimators,parent=models,value='RF')



            #adding conditions and HPs
            confs.add_hyperparameters([models,Kneighbors,rc,nestimators,kernel])
            confs.add_conditions([useks,userc,usekernel,useEst])
        elif self.Problemtype=='Regression':
            models=Categorical('Models',self.Models)
            #linear regression parameters
            #lasso and ridge regression parameters
            alpha=Float('alpha',(0.01,100))
            
            #random forest and XGboost parameters
            nestimators=Integer('n_estimators',(1,20),default=10)
            #dependencies 
            usealpha=InCondition(child=alpha,parent=models,values=['Lasso','Ridge'])
            useEst=InCondition(child=nestimators,parent=models,values=['RF','XGboost'])
            #adding conditions and HPs
            confs.add_hyperparameters([models,alpha,nestimators])
            confs.add_conditions([usealpha,useEst])
        elif self.Problemtype=='TimeSeries':
            # ['arima',sarima,'prophet']
            models=Categorical("Models",self.Models)
            
            p=Integer('p',(0,3))
            q=Integer('q',(0,3))
            d=Integer('d',(0,3))
            SarimaP=Integer('sp',(0,3))
            sarimaq=Integer('sq',(0,3))
            sarimad=Integer('sd',(0,3))
            
            usecondip=EqualsCondition(child=SarimaP,parent=models,value='Sarima')
            usecondiq=EqualsCondition(child=sarimaq,parent=models,value='Sarima')
            usecondid=EqualsCondition(child=sarimad,parent=models,value='Sarima')
            confs.add_hyperparameters([models,p,q,d,SarimaP,sarimaq,sarimad])
            confs.add_conditions([usecondip,usecondiq,usecondid])
            
        return confs
            
        
    def train(self,config:Configuration,seed: int=0):
        start_time=time.time()
        config_dict=config.get_dictionary()
        model=config_dict['Models']
        print(f"config_dict:{config_dict}")
        if self.Problemtype=='Classification':
            return self.classification(config_dict,start_time)
        elif self.Problemtype=='Regression':
            return self.regression(config_dict,start_time)
        elif self.Problemtype=='TimeSeries':
            return self.timeser(config_dict)
    def classification(self,configDict,start_time):
        model=configDict['Models']
        if model=='KNN':
            Classifier=KNeighborsClassifier(n_neighbors=configDict['Ks'])
        elif model=='LR':
            Classifier=LogisticRegression(C=configDict['regularizationStre'])
        elif model=='RF':
            Classifier=RandomForestClassifier(n_estimators=configDict['n_estimators'],random_state=42)
        elif model=='SVC':
            Classifier=SVC(C=configDict['regularizationStre'],kernel=configDict['kernel'])
        print(f"the type of the classifier is: {type(Classifier)}")
        Classifier.fit(self.X_train,self.Y_train)
        y_pred = Classifier.predict(self.X_test)
        loss=1-accuracy_score(self.Y_test,y_pred)
        print("the loss is: ",loss)
        return {'loss':loss,'time':time.time()-start_time}
    def regression(self, configDict,start_time):
            model=configDict['Models']
            if model=='LR':
                regressor=LinearRegression()
            elif model=='Lasso':
                regressor=Lasso(alpha=configDict['alpha'])
            elif model=='Ridge':
                regressor=Ridge(alpha=configDict['alpha'])
            elif model=='RF':
                regressor=RandomForestRegressor(n_estimators=configDict['n_estimators'],random_state=42)
            elif model=='XGboost':
                regressor=XGBRegressor(n_estimators=configDict['n_estimators'],random_state=42)
            regressor.fit(self.X_train,self.Y_train)
            y_pred = regressor.predict(self.X_test)
            mse = mean_squared_error(self.Y_test, y_pred)
            print("Mean Squared Error:", mse)
            return {'loss':mse,'time':time.time()-start_time}
    def timeser(self,configDict):
        model=configDict['Models']
        if model=='Arima':
            return Arima(self.Y_train,self.Y_test,p=configDict['p'],d=configDict['d'],q=configDict['q'],freq=self.freq)
        elif model=='Sarima':
            return 10000
            
            


In [63]:
from enum import Enum

class ProblemType(Enum):
    CLASSIFICATION = 'classification'
    REGRESSION = 'regression'
    TIME_SERIES = 'time series'
    UNBALANCED = 'unbalanced'

class Facade:
    def __init__(self, problem_type,Models,X_train,X_test,Y_train,Y_test,freq="d",m=7):
        if isinstance(problem_type, ProblemType):
            self.problem_type = problem_type
            self.models=Models
            self.X_train=X_train
            self.X_test=X_test
            self.Y_train=Y_train
            self.Y_test=Y_test
            self.freq=freq
            self.m=m
        else:
            raise ValueError("problem_type must be a ProblemType Enum")
    def chooseFacade(self):
        if self.problem_type==ProblemType.CLASSIFICATION:
            return  self.ClassificationFacade()
        elif self.problem_type==ProblemType.REGRESSION:
            return self.RegressionFacade()
        elif self.problem_type==ProblemType.TIME_SERIES:
            return self.TimeSeriesFacade()
        elif self.problem_type==ProblemType.UNBALANCED:
            return self.UnbalancedFacade()
    def ClassificationFacade(self):
        classifier=Models(self.models,'Classification',self.X_train,self.Y_train,self.X_test,self.Y_test)
        scenario = Scenario(classifier.configspace(), deterministic=True,objectives=['loss','time'], n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, classifier.train,overwrite=True,callbacks=[CustomCallback()],
                                                multi_objective_algorithm=HyperparameterOptimizationFacade.get_multi_objective_algorithm(scenario,objective_weights=[2, 1]))
        incumbents = smac.optimize()
        for incumbent in incumbents:
            print(incumbent)
        return incumbents
    def RegressionFacade(self):
        Regressor=Models(self.models,'Regression',self.X_train,self.Y_train,self.X_test,self.Y_test)
        scenario = Scenario(Regressor.configspace(), deterministic=True,objectives=['loss','time'], n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, Regressor.train,overwrite=True,callbacks=[CustomCallback()],
                                                multi_objective_algorithm=HyperparameterOptimizationFacade.get_multi_objective_algorithm(scenario,objective_weights=[2, 1]))
        incumbents = smac.optimize()
        for incumbent in incumbents:
            print(incumbent)
        return incumbents
    def TimeSeriesFacade(self):
        timeclassifier=Models(self.models,'TimeSeries',self.X_train,self.Y_train,self.X_test,self.Y_test,freq=self.freq,m=self.m)
        scenario = Scenario(timeclassifier.configspace(), deterministic=True, n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, timeclassifier.train,overwrite=True,callbacks=[CustomCallback()])
        incumbent=smac.optimize()
        print(incumbent)
        return incumbent
        
    def UnbalancedFacade(self):
        pass

In [64]:
path = r"C:\Users\Alaa\Desktop\archive\daily-minimum-temperatures-in-me.csv"
df =pd.read_csv(path)
df.rename(columns={'Date': 'ds', 'Daily minimum temperatures': 'y'}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'], format='%m/%d/%Y')
df['y'] = df['y'].str.replace('[^0-9\.]', '', regex=True)
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] = df['y'].astype(float)
df.set_index('ds', inplace=True)
split_date = pd.to_datetime('1990-12-15')
train_data = df[df.index <= split_date]
test_data = df[df.index > split_date]
traindatax='lol'
traindatay=train_data['y']
test_datay=test_data['y']
test_datax='loll'


In [65]:
train_data

,y
ds,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-11,11.1
1990-12-12,14.0
1990-12-13,11.4


In [66]:
pd.DataFrame(traindatay)

,y
ds,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-11,11.1
1990-12-12,14.0
1990-12-13,11.4


In [67]:
test_datay

ds
1990-12-16    13.6
1990-12-17    13.9
1990-12-18    17.2
1990-12-19    14.7
1990-12-20    15.4
1990-12-21    13.1
1990-12-22    13.2
1990-12-23    13.9
1990-12-24    10.0
1990-12-25    12.9
1990-12-26    14.6
1990-12-27    14.0
1990-12-28    13.6
1990-12-29    13.5
1990-12-30    15.7
1990-12-31    13.0
Name: y, dtype: float64

In [82]:
print(Arima(pd.DataFrame(traindatay),pd.DataFrame(test_datay),p=1,d=1,q=2,freq='d'))

7.63395894253463


In [69]:
if __name__=="__main__":
    
    classincummbet=Facade(ProblemType.TIME_SERIES,['Arima','Sarima'],traindatax,test_datax,pd.DataFrame(traindatay),pd.DataFrame(test_datay))
    incumbent=classincummbet.chooseFacade()

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 70 to 25 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 25 initial design configurations and 0 additional configurations.
let's start the optimization
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
config_dict:{'Models': 'Arima', 'd': 2, 'p': 0, 'q': 0}
[INFO][abstract_intensifier.py:515] Added config 0963f6 as new incumbent because there are no incumbents yet.
the number of trials is: 1
config_dict:{'Models': 'Sarima', 'd': 0, 'p': 3, 'q': 3, 'sd': 0, 'sp': 3, 'sq': 0}
the number of trials is: 2
config_dict:{'Models': 'Sarima', 'd': 3, 'p': 1, 'q': 1, 'sd': 1, 'sp': 2, 'sq': 2}
the number of trials is: 3
config_dict:{'Models': 'Arima', 'd': 1, 'p': 2, 'q': 2}


C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()
C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()


[INFO][abstract_intensifier.py:594] Added config f1b465 and rejected config 0963f6 as incumbent because it is not better than the incumbents on 1 instances:
the number of trials is: 4
config_dict:{'Models': 'Arima', 'd': 3, 'p': 3, 'q': 2}


C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()
C:\Users\Alaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


the number of trials is: 5
config_dict:{'Models': 'Sarima', 'd': 1, 'p': 0, 'q': 1, 'sd': 1, 'sp': 3, 'sq': 1}
the number of trials is: 6
config_dict:{'Models': 'Sarima', 'd': 2, 'p': 2, 'q': 3, 'sd': 0, 'sp': 2, 'sq': 3}
the number of trials is: 7
config_dict:{'Models': 'Arima', 'd': 0, 'p': 1, 'q': 0}


...

C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()
C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()


config_dict:{'Models': 'Sarima', 'd': 2, 'p': 1, 'q': 2, 'sd': 1, 'sp': 1, 'sq': 0}
the number of trials is: 96
config_dict:{'Models': 'Sarima', 'd': 2, 'p': 1, 'q': 0, 'sd': 3, 'sp': 1, 'sq': 0}
the number of trials is: 97
config_dict:{'Models': 'Arima', 'd': 0, 'p': 2, 'q': 0}
the number of trials is: 98


C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()


config_dict:{'Models': 'Sarima', 'd': 3, 'p': 0, 'q': 3, 'sd': 2, 'sp': 2, 'sq': 2}
the number of trials is: 99
config_dict:{'Models': 'Sarima', 'd': 1, 'p': 3, 'q': 2, 'sd': 0, 'sp': 3, 'sq': 0}
the number of trials is: 100
the incumbent is: {'Models': 'Arima', 'd': 0, 'p': 3, 'q': 3}
the incumbent loss is: 7.592214477774151
let's stop the optimization at trial 100
[INFO][smbo.py:224] A callback returned False. Abort is requested.
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:327] Configuration budget is exhausted:
[INFO][smbo.py:328] --- Remaining wallclock time: inf
[INFO][smbo.py:329] --- Remaining cpu time: inf
[INFO][smbo.py:330] --- Remaining trials: 0
Configuration(values={
  'Models': 'Arima',
  'd': 0,
  'p': 3,
  'q': 3,
})


C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:103: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  config_dict=config.get_dictionary()
C:\Users\Alaa\AppData\Local\Temp\ipykernel_19228\4216220568.py:25: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  print(f"the incumbent is: {incumbent.get_dictionary()}")


In [ ]:

if __name__ =="__main__":
#test classifcation
    classincummbet=Facade(ProblemType.CLASSIFICATION,['KNN','LR','RF','SVC'],X_train_raw,X_test,y_train,y_test)
    classincummbet.chooseFacade()
#     print(classincummbet)

    


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data[:, 1:]
y = iris.data[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
if __name__=="__main__":
    regreincumbent=Facade(ProblemType.REGRESSION,['LR','Lasso','Ridge','RF','XGboost'],X_train,X_test,y_train,y_test)
    regreincumbent.chooseFacade()

In [ ]:
regressor=XGBRegressor(n_estimators=10,random_state=42)
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
regressor=RandomForestRegressor(n_estimators=19)
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
Classifier=RandomForestClassifier(n_estimators=19)
Classifier.fit(X_train_raw,y_train)
y_pred = Classifier.predict(X_test)
loss=1-accuracy_score(y_test,y_pred)
print("the loss is: ",loss)